<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# Improving Instruction-Data Via Reflection-Tuning Using GPT-4
# 使用 GPT-4 通过反思调优改进指令数据

- This notebook uses OpenAI's GPT-4 API to implement the dataset refinement process from the [Reflection-Tuning: Data Recycling Improves LLM Instruction-Tuning](https://arxiv.org/abs/2310.11716) paper
- 本笔记本使用OpenAI的GPT-4 API来实现来自[Reflection-Tuning: 数据回收改进LLM指令调优](https://arxiv.org/abs/2310.11716)论文的数据集优化过程

![](https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/reflection-tuning/reflection-tuning.webp)

- In the original paper, the researchers refined the [Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca) and [WizardLM](https://huggingface.co/datasets/WizardLMTeam/WizardLM_evol_instruct_70k) instruction-finetuning datasets; in this notebook, we refine the [instruction-dataset used in chapter 7](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch07/01_main-chapter-code/instruction-data.json) (however, since it has the same format as Alpaca, the same code works with the Alpaca dataset as well)
- 在原论文中，研究人员优化了[Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca)和[WizardLM](https://huggingface.co/datasets/WizardLMTeam/WizardLM_evol_instruct_70k)指令微调数据集；在本笔记本中，我们优化[第7章使用的指令数据集](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch07/01_main-chapter-code/instruction-data.json)（由于它与Alpaca具有相同的格式，因此相同的代码也适用于Alpaca数据集）

- The expected dataset format is as follows:

```python
    {
        "instruction": "Edit the following sentence for grammar.",
        "input": "He go to the park every day.",
        "output": "He goes to the park every day."
    },
    {
        "instruction": "Convert 45 kilometers to meters.",
        "input": "",
        "output": "45 kilometers is 45000 meters."
    },
```

> Please note that this notebook reproduces the approach from the paper in which the authors used the GPT API to enhance existing datasets. However, it's important to be aware that GPT API-generated data may not be used to develop models that compete with OpenAI, as specified in the [OpenAI Terms of Use](https://openai.com/policies/row-terms-of-use/): "What you cannot do... Use Output to develop models that compete with OpenAI."
> 请注意，本笔记本复现了论文中作者使用 GPT API 增强现有数据集的方法。但是，请务必注意，根据 [OpenAI 使用条款](https://openai.com/policies/row-terms-of-use/) 规定，GPT API 生成的数据不能用于开发与 OpenAI 竞争的模型："禁止事项...使用输出开发与 OpenAI 竞争的模型。"

You can find a relevant discussion [here](https://www.reddit.com/r/LocalLLaMA/comments/17vbg1f/does_openai_tos_prohibit_generating_datasets_for/)).
你可以在[这里](https://www.reddit.com/r/LocalLLaMA/comments/17vbg1f/does_openai_tos_prohibit_generating_datasets_for/))找到相关讨论。

In [1]:
# pip install -r requirements-extra.txt

In [2]:
from importlib.metadata import version

pkgs = [
    "openai",  # OpenAI API
    "tqdm",    # Progress bar
]

for p in pkgs:
    print(f"{p} version: {version(p)}")

openai version: 1.30.3
tqdm version: 4.66.4


## Test OpenAI API
## 测试 OpenAI API

- First, let's test if the OpenAI API is correctly set up
- 首先，让我们测试OpenAI API是否正确设置
- If you don't have an account yet, you need to create one at https://platform.openai.com/
- 如果你还没有账号，需要在 https://platform.openai.com/ 创建一个
- Note that you will also have to transfer some funds to your account as the GPT-4 API is not free (see https://platform.openai.com/settings/organization/billing/overview)
- 请注意，由于GPT-4 API不是免费的，你还需要向账户转入资金（参见 https://platform.openai.com/settings/organization/billing/overview）
- Running the code exactly as it appears in this notebook costs about \$0.03 (3 cents) with GPT-4o-mini as of this writing
- 按照本笔记本中的代码运行，使用GPT-4o-mini目前大约需要花费0.03美元（3美分）
- Applying the two methodologies above to all 1100 entries in the chapter 7 instruction dataset costs about \$0.60 (60 cents)
- 将上述两种方法应用于第7章指令数据集的所有1100个条目，大约需要花费0.60美元（60美分）

- First, we need to provide our OpenAI API secret key, which can be found at https://platform.openai.com/api-keys
- 首先，我们需要提供 OpenAI API 密钥，可以在 https://platform.openai.com/api-keys 找到
- Make sure not to share this key with anyone
- 请确保不要与任何人分享此密钥
- Add this secret key (`"sk-..."`) to the `config.json` file in this folder
- 将此密钥 (`"sk-..."`) 添加到此文件夹中的 `config.json` 文件中

In [3]:
# 导入所需的库
import json
from openai import OpenAI

# 从 JSON 文件加载 API 密钥
# 确保将 "sk-..." 替换为从 https://platform.openai.com/api-keys 获取的实际 API 密钥
with open("config.json", "r") as config_file:
    config = json.load(config_file)
    api_key = config["OPENAI_API_KEY"]

# 初始化 OpenAI 客户端
client = OpenAI(api_key=api_key)

- First, let's try the API with a simple example to make sure it works as intended:
- 首先，让我们用一个简单的例子来测试API，确保它能按预期工作：

In [4]:
# 定义一个函数来运行ChatGPT API调用
def run_chatgpt(prompt, client, model="gpt-4o-mini", system_prompt=None):
    # 定义一个空的消息列表用于存储对话内容
    messages = []
    
    # 如果提供了系统提示词，将其添加到消息列表中
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # 将用户的提示词添加到消息列表中
    messages.append({"role": "user", "content": prompt})

    # 调用OpenAI API创建聊天完成
    response = client.chat.completions.create(
        model=model,        # 使用指定的模型
        messages=messages,  # 传入消息列表
        temperature=0.0,    # 设置温度为0以获得确定性输出
        seed=123,          # 设置随机种子以保证可重复性
    )
    
    # 返回模型的响应文本
    return response.choices[0].message.content


# 定义测试用的提示词
prompt = f"Respond with 'hello world' if you got this message."
# 调用函数测试API
run_chatgpt(prompt, client)

'hello world'

## Load JSON Entries
## 加载 JSON 条目

- Next, let's load and process the instruction dataset
- 接下来,让我们加载和处理指令数据集
- Here, we assume that we saved the test dataset and the model responses as a JSON file that we can load as follows:
- 在这里,我们假设已经将测试数据集和模型响应保存为JSON文件,我们可以按如下方式加载:

In [5]:
# 导入pathlib库中的Path类用于处理文件路径
from pathlib import Path


# 使用Path构建JSON文件的路径
json_file = Path("..") / "01_main-chapter-code" / "instruction-data.json"

# 打开并读取JSON文件的内容
with open(json_file, "r") as file:
    json_data = json.load(file)

# 打印数据集中的条目数量
print("Number of entries:", len(json_data))

Number of entries: 1100


- Let's print one of the dataset entries to see its structure:
- 让我们打印一个数据集条目来查看其结构:

In [6]:
# 从pprint模块导入pp函数并重命名为pprint
from pprint import pp as pprint

# 使用pprint函数打印json_data中的第一个条目
pprint(json_data[0])

{'instruction': 'Evaluate the following phrase by transforming it into the '
                'spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the '
           'correct spelling is "friend".'}


## Improve Instructions
## 改进指令

- The Reflection-Tuning authors shared two approaches: (1) improving the instructions and (2) improving the responses
- Reflection-Tuning 作者分享了两种方法:(1)改进指令和(2)改进响应

- Let's begin by improving the instructions in a given dataset  
- 让我们从改进给定数据集中的指令开始

- Below is a small utility function from the [Reflection-Tuning repository](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py) to format the inputs to the GPT-4 model for this dataset refinement
- 下面是来自 [Reflection-Tuning 仓库](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py) 的一个小工具函数,用于格式化输入到 GPT-4 模型以进行数据集优化

In [7]:
def instr_prompt_no_input(ins, outp):
    """
    生成用于评估指令质量的提示模板
    
    参数:
        ins: 原始指令文本
        outp: 原始回答文本
        
    返回:
        sys_prompt: 系统提示语
        prompt: 完整的提示模板
    """
    # 定义系统提示语,说明助手的角色
    sys_prompt = "You are a helpful, precise but picky assistant for checking the quality of a given instruction."
    
    # 定义提示模板的基本结构
    prompt_template = "[Instruction]\n{ins}\n\n[The Start of Answer]\n{outp}\n\n[The End of Answer]\n\n[System]\n{criteria}\n\n"
    
    # 定义评估标准,包含三个主要部分:
    # 1. 分析指令的不足之处
    # 2. 生成新的更好的指令
    # 3. 回答新生成的指令
    criteria = "We would like you to answer several questions related to the quality of a given instruction. \n" + \
                "1. Why this instruction is not good? First analyse the instruction based on Complexity of the Topic, Level of Detail Required, Knowledge Required, Ambiguity of the Instruction and Logical Reasoning or Problem-Solving Involved. \n" + \
                "Then analyse why this answer is not good for the given instruction? Analyse based on the Helpfulness, Relevance, Accuracy and Level of Details. \n" + \
                "Finally analyse why this bad instruction lead to a bad answer. " +\
                "2. Based on the reason you provided, generate a new and complete instruction which is complex and difficult to answer directly. " + \
                "Make sure the new instruction is relevent but independent to the original instruction, which can be answered without knowing the original instruction, put the new instruction in the format of [New Instruction] your instruction [End]" +\
                "3. Answer the newly generated instruction as detailed as possible, in the format of [New Answer] your answer [End] \n"
    
    # 使用format方法填充模板
    prompt = prompt_template.format(
        ins=ins, outp=outp, criteria=criteria
    )
    
    return sys_prompt, prompt

- To see how it works, consider the dataset entry, `json_data[2]`
- 为了看看它是如何工作的，让我们来看看数据集条目 `json_data[2]`

In [8]:
# 打印json_data中的第三个条目
pprint(json_data[2])

{'instruction': 'Convert 45 kilometers to meters.',
 'input': '',
 'output': '45 kilometers is 45000 meters.'}


- We can refine the instruction as follows, using `instr_prompt_no_input` function defined above:
- 我们可以使用上面定义的 `instr_prompt_no_input` 函数来优化指令,具体如下:

In [ ]:
# 从数据集中获取第3个样本
entry = json_data[2]

# 使用instr_prompt_no_input函数生成系统提示和完整提示
system_prompt, prompt = instr_prompt_no_input(ins=entry["instruction"], outp=entry["output"])

# 调用ChatGPT API获取输出结果
output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)

# 打印输出结果
print(output)

1. **Analysis of the Instruction:**

   - **Complexity of the Topic:** The topic of converting kilometers to meters is relatively simple and straightforward, as it involves basic unit conversion.
   - **Level of Detail Required:** The instruction does not require much detail; it simply asks for a conversion without any additional context or explanation.
   - **Knowledge Required:** Basic knowledge of metric units and their conversions is required, which is common knowledge.
   - **Ambiguity of the Instruction:** The instruction is clear and unambiguous; it specifies exactly what needs to be converted.
   - **Logical Reasoning or Problem-Solving Involved:** There is minimal logical reasoning involved, as the conversion factor (1 kilometer = 1000 meters) is a standard fact.

   **Analysis of the Answer:**

   - **Helpfulness:** The answer is helpful in that it provides the correct conversion.
   - **Relevance:** The answer is relevant to the instruction, as it directly addresses the conv

- The response is very verbose, which is useful for analysis purposes; also, it helps the GPT-4 model to make improvements via the chain-of-thought prompting approach
- 响应非常冗长，这对分析目的很有用；同时，它也帮助GPT-4模型通过思维链提示方法进行改进

- However, to construct the improved dataset, we are actually only interested in new instructions and outputs, not the analyses
- 然而，为了构建改进的数据集，我们实际上只对新的指令和输出感兴趣，而不是分析过程

- We can use the following utility code from the [Reflection-Tuning repository](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py) to extract the model's improved instructions and outputs
- 我们可以使用来自[Reflection-Tuning仓库](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py)的以下实用代码来提取模型改进后的指令和输出

In [ ]:
# 导入正则表达式模块
import re

# 从文本中提取指令部分的函数
def extract_ins(text, no_input=True):
    # 检查文本中是否包含[New Instruction]标记
    if '[New Instruction]' in text:
        # 如果包含,使用对应的正则表达式模式
        pattern = r'(\[New Instruction\])(.*?)(\[End\]|\[New Answer\]|New Answer:)'
    else:
        # 否则使用另一个正则表达式模式
        pattern = r'(New Instruction:)(.*?)(\[End\]|\[New Answer\]|New Answer:)'
    # 使用正则表达式查找所有匹配项
    segments = re.findall(pattern, text, re.DOTALL)
    # 如果没有找到匹配项,返回空字符串
    if len(segments) == 0:
        seg_ins = ''
    else:
        # 否则获取第一个匹配项的第二个分组并去除首尾空白
        seg_ins = segments[0][1].strip()
    # 如果文本以"\n\n3."结尾,去除这部分
    if seg_ins.endswith("\n\n3."):
        seg_ins = seg_ins[:-4]
    return seg_ins

# 从文本中提取输出部分的函数
def extract_oup(text, no_input=True):
    # 检查文本中是否包含[New Answer]标记
    if '[New Answer]' in text:
        # 如果包含,使用对应的正则表达式模式
        pattern = r'(\[New Answer\])(.*?)(\[End\]|$)'
    else:
        # 否则使用另一个正则表达式模式
        pattern = r'(New Answer:)(.*?)(\[End\]|$)'
        # 注释掉的备选模式
        # pattern = r'(\[New Answer\]|New Answer:)(.*?)(\[End\]|$)'
    # 使用正则表达式查找所有匹配项
    segments = re.findall(pattern, text, re.DOTALL)
    # 如果没有找到匹配项,返回空字符串
    if len(segments) == 0:
        seg_oup = ''
    else:
        # 否则获取第一个匹配项的第二个分组并去除首尾空白
        seg_oup = segments[0][1].strip()
    return seg_oup

# 提取完整指令(包含输入和输出)的函数
def extract_instruction(text):
    # 如果输入文本为空,返回空列表
    if text == '':
        return []
    # 提取指令部分
    seg_ins = extract_ins(text, no_input=True)
    # 提取输出部分
    seg_oup = extract_oup(text, no_input=True)
    # 返回包含指令和输出的列表
    return [seg_ins, seg_oup]

- Let's use these utility functions to extract the improved instruction and response from the lengthy GPT-4 output generated earlier:
- 让我们使用这些工具函数从之前生成的冗长的GPT-4输出中提取改进后的指令和响应:

In [ ]:
# 从GPT-4输出中提取改进后的指令和回答
new_instr, new_outp = extract_instruction(output)

In [ ]:
# 打印提取出的新指令
print(new_instr)

Explain the significance of the metric system in global trade and provide examples of how unit conversions can impact international business transactions.


In [ ]:
# 打印从GPT-4输出中提取的改进后的回答
print(new_outp)

The metric system, also known as the International System of Units (SI), is a decimal-based system of measurement that is used globally. Its significance in global trade lies in its standardization, which facilitates international communication and commerce. 

   One of the primary advantages of the metric system is that it is universally recognized, which reduces confusion and errors in measurement. For example, when a company in the United States imports goods from Europe, the specifications for those goods are often provided in metric units. If the U.S. company is accustomed to using imperial units (like inches or pounds), they must convert these measurements to ensure compatibility. 

   Unit conversions can significantly impact international business transactions. For instance, if a manufacturer orders 100 kilograms of a product but mistakenly interprets it as 100 pounds, they will receive a much smaller quantity than intended, leading to production delays and financial losses. 



- Note that the instruction-refinement is currently only implemented for dataset entries that don't have an `"input"` field
- 请注意,指令优化目前仅针对没有"input"字段的数据集条目实现

## Improve Responses
## 改进回答

- In a similar fashion, we can also apply the Reflection-Tuning refinement process specifically to the dataset responses (i.e., "output" fields)
- 类似地,我们也可以专门针对数据集的响应(即"output"字段)应用Reflection-Tuning优化流程
- Below are two small utility functions from the [Reflection-Tuning repository](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py) to format the inputs to the GPT-4 model for dataset refinement
- 以下是来自[Reflection-Tuning仓库](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py)的两个小工具函数,用于格式化输入到GPT-4模型以进行数据集优化

In [ ]:
# 生成不带输入的响应提示的函数
def res_gen_prompt_no_input(ins, outp):

    # 定义系统提示,说明助手的角色是帮助检查回答质量
    sys_prompt = "You are a helpful, precise but picky assistant for checking the quality of the answer to a given instruction."
    # 定义提示模板,包含指令、答案和系统部分
    prompt_template = "[Instruction]\n{ins}\n\n[The Start of Answer]\n{outp}\n\n[The End of Answer]\n\n[System]\n{criteria}\n\n"
    # 定义评估标准,包括分析答案质量和生成更好答案的要求
    criteria = "We would like you to answer several questions related to the quality of the answer to the given instruction. \n" + \
                "1. Why this answer is not good for the given instruction? Analyse based on the Helpfulness, Relevance, Accuracy and Level of Details. \n" + \
                "2. Based on the reason you provided, generate a better answer, new and complete, as detailed as possible, in the format of [Better Answer] your answer [End] \n" 
    # 使用提供的参数格式化提示模板
    prompt = prompt_template.format(
        ins=ins, outp=outp, criteria=criteria
    )
    # 返回系统提示和格式化后的提示
    return sys_prompt, prompt


# 生成带输入的响应提示的函数
def res_gen_prompt_input(ins, inp, outp):

    # 定义系统提示,说明助手的角色是帮助检查带输入的回答质量
    sys_prompt = "You are a helpful and precise assistant for checking the quality of the answer to a given instruction and its input."
    # 定义提示模板,包含指令、输入、答案和系统部分
    prompt_template = "[Instruction]\n{ins}\n\n[The Start of Input]\n{inp}\n\n[The End of Input]\n\n[The Start of Answer]\n{outp}\n\n[The End of Answer]\n\n[System]\n{criteria}\n\n"
    # 定义评估标准,包括分析答案质量和生成更好答案的要求
    criteria = "We would like you to answer several questions related to the quality of the answer to the given instruction and corresponding input. \n" + \
                "1. Why this answer is not good for the given instruction and corresponding input? Analyse based on the Helpfulness, Relevance, Accuracy and Level of Details. \n" + \
                "2. Based on the reason you provided, generate a better answer, new and complete, as detailed as possible, in the format of [Better Answer] your answer [End] \n" 
    # 使用提供的参数格式化提示模板
    prompt = prompt_template.format(
        ins=ins, inp=inp, outp=outp, criteria=criteria
    )
    # 返回系统提示和格式化后的提示
    return sys_prompt, prompt

- Again, let's apply it to one of the dataset entries to see how it works, generating the improved response:
- 让我们再次将其应用于其中一个数据集条目,看看它是如何工作的,生成改进后的回答:

In [15]:
# 获取数据集中的第3个条目
entry = json_data[2]

# 使用没有输入字段的提示模板生成系统提示和用户提示
system_prompt, prompt = res_gen_prompt_no_input(ins=entry["instruction"], outp=entry["output"])

# 调用ChatGPT API获取改进的回答
output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)

# 打印ChatGPT的输出结果
print(output)

1. The answer provided is not good for the given instruction for several reasons:

- **Helpfulness**: While the answer does provide the correct conversion, it lacks any explanation or context. A more helpful answer would include a brief explanation of the conversion process, which would aid understanding.

- **Relevance**: The answer is relevant in that it addresses the instruction to convert kilometers to meters, but it could be more relevant by including the conversion factor used (1 kilometer = 1000 meters).

- **Accuracy**: The answer is accurate in terms of the numerical conversion (45 kilometers = 45000 meters). However, it could be misleading if the reader does not understand how the conversion was derived.

- **Level of Details**: The answer is very brief and lacks detail. A more detailed response would include the conversion factor and a step-by-step explanation of how the conversion is performed.

2. [Better Answer] To convert kilometers to meters, you can use the conversion 

- 如上所示,响应包含了对原始响应的分析;我们可以使用以下来自[Reflection-Tuning仓库](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py)的工具函数来提取新的响应
- As we can see above, the response includes an analysis of the original response; we can extract the new response using the following utility function from the [Reflection-Tuning repository](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py)

In [16]:
# 定义一个函数用于从文本中提取回答内容
def extract_response(text):
    # 如果文本中包含多个"Better Answer"标记
    if text.count('[Better Answer]') >= 2:
        # 定义正则表达式模式来匹配多个回答片段
        pattern = r'\[(Better Answer)\](.*?)(\[End\]|\[Better Answer\]|$)'
        # 使用正则表达式查找所有匹配的片段
        segments = re.findall(pattern, text, re.DOTALL)
    else:
        # 如果只有一个"Better Answer"标记
        # 定义正则表达式模式来匹配单个回答片段
        pattern = r'\[(Better Answer)\](.*?)(\[End\]|End|$)'
        # 使用正则表达式查找匹配的片段
        segments = re.findall(pattern, text, re.DOTALL)
    # 返回所有找到的回答内容(去除首尾空白)
    return [segment[1].strip() for segment in segments]

In [17]:
# 从输出中提取第一个改进后的回答
response = extract_response(output)[0]
# 打印改进后的回答
print(response)

To convert kilometers to meters, you can use the conversion factor that 1 kilometer is equal to 1000 meters. Therefore, to convert 45 kilometers to meters, you multiply 45 by 1000. 

So, 45 kilometers × 1000 meters/kilometer = 45000 meters. 

Thus, 45 kilometers is equal to 45000 meters.


## Improving the Dataset
## 改进数据集

- 现在,让我们将指令反思和响应反思技术应用到实际数据集
- Now, let's apply the instruction-reflection and response-reflection techniques to the actual dataset
- 注意:这里我们仅将其应用于一个小型数据子集用于演示目的;如果要应用到整个数据集,需要更改
- Note: we only apply it to a small data subset here for demo purposes; to apply it to the whole dataset, change

```python
data_to_process = json_data[:3]
```

to

```python
data_to_process = json_data
```

### Reflect instructions
### 反思指令

- The following code applies the Reflection-Tuning methodology for dataset refinement to the instructions in the original dataset
- 以下代码将反思调优方法应用于原始数据集中的指令进行优化

In [18]:
# 仅处理前3条数据用于演示
data_to_process = json_data[:3]

In [19]:
# 导入tqdm库用于显示进度条
from tqdm import tqdm


# 定义反思指令的函数,接收json数据和client作为参数
def reflect_instructions(json_data, client):
    # 初始化新的json数据列表
    new_json_data = [] 
    
    # 使用tqdm遍历json数据条目并显示进度
    for entry in tqdm(json_data):
        
        # 如果条目没有输入内容
        if not entry["input"]:
            # 根据指令和输出生成系统提示和用户提示
            system_prompt, prompt = instr_prompt_no_input(ins=entry["instruction"], outp=entry["output"])
            # 调用ChatGPT获取输出
            output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)
            # 从输出中提取新的指令和输出
            new_instr, new_outp = extract_instruction(output)
            # 创建新的数据条目
            new_entry = {"instruction": new_instr, "input": "", "output": new_outp}
            # 将新条目添加到结果列表
            new_json_data.append(new_entry)
        else:
            # 如果有输入内容,直接添加原条目
            new_json_data.append(entry)

    # 返回处理后的新数据
    return new_json_data

In [20]:
# 只处理前3条数据进行测试
data_to_process = json_data[:3]

# 对指令数据进行反思优化,生成新的数据集
new_json_data = reflect_instructions(data_to_process, client)

100%|█████████████████████████████████████████████| 3/3 [00:06<00:00,  2.17s/it]


In [21]:
# 遍历前3个数据项并打印
# 使用pprint进行格式化输出
# 每个数据项之间添加两个空行
for i in new_json_data[:3]:
    pprint(i)
    print("\n\n")

{'instruction': 'Evaluate the following phrase by transforming it into the '
                'spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the '
           'correct spelling is "friend".'}



{'instruction': 'Edit the following sentence for grammar.',
 'input': 'He go to the park every day.',
 'output': 'He goes to the park every day.'}



{'instruction': 'Explain the significance of understanding metric conversions '
                'in scientific research, and provide an example of how a '
                'miscalculation in unit conversion could impact experimental '
                'results.',
 'input': '',
 'output': 'Understanding metric conversions is crucial in scientific research '
           'because accurate measurements are fundamental to the validity of '
           'experimental results. The metric system is widely used in '
           'scientific disciplines due to its ease of use and universal '
    

- Let's save the new dataset:
- 让我们保存新的数据集：

In [22]:
# 将处理后的数据保存到JSON文件中
# 使用indent=4进行格式化输出，提高可读性
with open("instruction-reflected.json", "w") as file:
    json.dump(new_json_data, file, indent=4)

### Reflect responses
### 反思响应

- Let's now do the same for the response-reflection:
- 现在让我们对响应内容做同样的反思处理：

In [23]:
# 只处理前3条数据进行测试
data_to_process = json_data[:3]

In [24]:
# 定义一个函数用于反思和改进响应内容
def reflect_responses(json_data, client):
    # 初始化一个空列表用于存储新的数据
    new_json_data = [] 
    
    # 使用tqdm显示进度条,遍历输入的json数据
    for entry in tqdm(json_data):
        
        # 如果数据项没有input字段
        if not entry["input"]:
            # 使用无输入的提示模板生成system_prompt和prompt
            system_prompt, prompt = res_gen_prompt_no_input(ins=entry["instruction"], outp=entry["output"])
            # 调用ChatGPT API获取输出
            output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)
            # 从输出中提取响应内容
            new_response = extract_response(output)

            # 如果没有提取到新响应,使用原始输出
            if not len(new_response):
                new_response = entry["output"]
                      
            # 创建新的数据项,包含原始指令、空输入和新响应
            new_entry = {"instruction": entry["instruction"], "input": "", "output": new_response[0]}
            # 将新数据项添加到结果列表中
            new_json_data.append(new_entry)

        # 如果数据项有input字段
        else:
            # 使用带输入的提示模板生成system_prompt和prompt
            system_prompt, prompt = res_gen_prompt_input(ins=entry["instruction"], inp=entry["input"], outp=entry["output"])
            # 调用ChatGPT API获取输出
            output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)
            # 从输出中提取响应内容
            new_response = extract_response(output)

            # 如果没有提取到新响应,使用原始输出
            if not len(new_response):
                new_response = entry["output"]

            # 创建新的数据项,包含原始指令、输入和新响应
            new_entry = {"instruction": entry["instruction"], "input": entry["input"], "output": new_response[0]}
            # 将新数据项添加到结果列表中
            new_json_data.append(new_entry)

    # 返回包含所有新数据项的列表
    return new_json_data

In [25]:
# 调用reflect_responses函数处理数据,生成改进后的响应
# 参数:
# - data_to_process: 要处理的原始数据
# - client: ChatGPT API客户端
# 返回包含改进响应的新数据集
new_json_data = reflect_responses(data_to_process, client)

100%|█████████████████████████████████████████████| 3/3 [00:07<00:00,  2.40s/it]


In [26]:
# 遍历前3个数据项并打印
for i in new_json_data[:3]:
    # 使用pprint美化打印每个数据项
    pprint(i)
    # 打印两个空行作为分隔
    print("\n\n")

{'instruction': 'Evaluate the following phrase by transforming it into the '
                'spelling given.',
 'input': 'freind --> friend',
 'output': 'The input phrase "freind" contains a spelling error. The correct '
           'transformation of the word is as follows: "freind" should be '
           'corrected to "friend." Therefore, the correct spelling is '
           '"friend."'}



{'instruction': 'Edit the following sentence for grammar.',
 'input': 'He go to the park every day.',
 'output': 'The original sentence "He go to the park every day" contains a '
           'grammatical error in the verb form. The correct form should be "He '
           'goes to the park every day." This is because the subject "He" is '
           'third person singular, and in English, the verb "to go" changes to '
           '"goes" when used with third person singular subjects. Therefore, '
           'the corrected sentence is grammatically accurate and maintains the '
           'original mea

- Let's save the new dataset:
- 让我们保存新的数据集：

In [27]:
# 将新的数据集保存到JSON文件中
# 使用indent=4进行格式化输出,使文件更易读
with open("response-reflected.json", "w") as file:
    json.dump(new_json_data, file, indent=4)

## Creating Improved Instruction Data
## 创建改进的指令数据

- Applying the two methodologies above to all 1100 entries in the chapter 7 instruction dataset costs about \$0.60 (60 cents)
- 将上述两种方法应用于第7章指令数据集中的所有1100个条目的成本约为0.60美元(60美分)
- To avoid bloating the GitHub repository with dataset files, the resulting dataset files are available from Google Drive:
- 为了避免数据集文件使GitHub仓库变得臃肿，生成的数据集文件可从Google Drive获取：
  - [instruction-reflected.json](https://drive.google.com/file/d/1c1QnuTdt9nP1u51vBn4_b05mWR_ZNGBv/view?usp=sharing)
  - [response-reflected.json](https://drive.google.com/file/d/1RNckTZ2ELcdUoJtaylao6NvyZPMtNv1v/view?usp=sharing)